# Построение системы анализа поведения клиентов сотового оператор

## Изучение данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('C:/documents/datasets/users_behavior.csv')

In [3]:
display(df)
df.info()
for i in df:
    display(i)
    display(df[i].describe())
display(df['is_ultra'].value_counts())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


'calls'

count    3214.000000
mean       63.038892
std        33.236368
min         0.000000
25%        40.000000
50%        62.000000
75%        82.000000
max       244.000000
Name: calls, dtype: float64

'minutes'

...

'mb_used'

count     3214.000000
mean     17207.673836
std       7570.968246
min          0.000000
25%      12491.902500
50%      16943.235000
75%      21424.700000
max      49745.730000
Name: mb_used, dtype: float64

'is_ultra'

count    3214.000000
mean        0.306472
std         0.461100
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: is_ultra, dtype: float64

0    2229
1     985
Name: is_ultra, dtype: int64

В датасете мы наблюдаем данные поведения 3214 клиентов. По кол-ву звонков в месяц от 0 до 244, в среднем 63 штуки. По минутам разговора от 0 до 1632 минут, в среднем 438 минут. Сообщения от 0 до 224, в среднем 38 штук. Интернет от 0 до 49745 мб, в среднем 17207 мб. Тарфиы - Ultra - 985 шт и Smart - 2229 шт.

<div class="alert-success"> 
<b>Комментарий ревьюера 👍</b>

Хороший первичный анализ данных

## Разделение данных на выборки

In [9]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']
features_train, features_1, target_train, target_1 = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_1, target_1, test_size=0.5, random_state=12345)
print('Обучающая:', features_train.shape, target_train.shape)
print('Валидационная:', features_valid.shape, target_valid.shape)
print('Тестовая:', features_test.shape, target_test.shape)

Обучающая: (1928, 4) (1928,)
Валидационная: (643, 4) (643,)
Тестовая: (643, 4) (643,)


Разбили датасет на 3 выборки - обучающая, валидационная и тестовая 60 на 20 на 20.

## Исследование моделей

1. модель дерева решений

In [10]:
best_model_tree = None
best_result_tree = 0
max_depth_tree = 0
criterion_tree = None
splitter_tree = None
for i in range(1, 6):
    for c in 'gini', 'entropy':
        for s in 'best', 'random':
            model_tree = DecisionTreeClassifier(random_state=12345, max_depth=i, criterion = c, splitter = s)
            model_tree.fit(features_train, target_train)
            predictions = model_tree.predict(features_valid)
            result = accuracy_score(target_valid, predictions)
            if result > best_result_tree:
                best_model_tree = model_tree
                best_result_tree = result
                max_depth_tree = i
                criterion_tree = c
                splitter_tree = s
        
print('Доля правильных ответов:', best_result_tree)
print('Максимальная глубина:', max_depth_tree)
print('Критерий:', criterion_tree)
print('Разделитель:', splitter_tree)

Доля правильных ответов: 0.7853810264385692
Максимальная глубина: 3
Критерий: gini
Разделитель: best


Мы обучили модель дерева решений и при проверке ее на валидационной выборке получили лучшую долю правильных ответов 78% при следующих гиперпараметрах: макимальная глубина - 3, критерий - gini,  разделитель - best.

2. Исследуем модель случайного леса

In [11]:
best_model_forest = None
best_result_forest = 0
n_estimators_forest = 0
min_samples_split_forest = 0
max_features_forest = None
for i in range(1, 100):
    for m in range (2, 4):
        for s in 'sqrt', 'log2':
            model_forest = RandomForestClassifier(random_state=12345, n_estimators=i, min_samples_split=m, max_features = s)
            model_forest.fit(features_train, target_train)
            predictions = model_forest.predict(features_valid)
            result = accuracy_score(target_valid, predictions)
            if result > best_result_forest:
                best_model_forest = model_forest
                best_result_forest = result
                n_estimators_forest = i
                min_samples_split_forest = m
                max_features_forest = s
        
print('Доля правильных ответов:', best_result_forest)
print('Количество деревьев:', n_estimators_forest)
print('Min число выборок:', min_samples_split_forest)
print('Кол-во функций:', max_features_forest)

Доля правильных ответов: 0.7947122861586314
Количество деревьев: 23
Min число выборок: 2
Кол-во функций: sqrt


Мы обучили модель случайного леса и при проверке ее на валидационной выборке получили лучшую долю правильных ответов 79% при следующих гиперпараметрах: количество деревьев - 23, min число выборок - 2,  кол-во функций - sqrt.

## Проверка модели на тестовой выборке

1. Проверяем модель дерева решений

In [13]:
predictions_tree = best_model_tree.predict(features_test)
result_tree = accuracy_score(target_test, predictions_tree)
display(result_tree)

0.7791601866251944

При проверке модели дерева решений с лучшими параметрами на тестовой выборке мы получили долю правильных ответов 78%. Праметр укладывается в ТЗ - болеее 75%

2. Проверяем модель случайного леса

In [14]:
predictions_forest = best_model_forest.predict(features_test)
result_forest = accuracy_score(target_test, predictions_forest)
display(result_forest)

0.7807153965785381

При проверке модели случайного леса c лучшими параметрами на тестовой выборке мы получили долю правильных ответов 78%. Праметр укладывается в ТЗ - болеее 75%. 

Поскольку скорость работы модели дерева решений быстрее случайного леса и на тестовой выборке доля правильных ответов у моделей сопостовимы - рекомендуется использовать модель деерва решений для данной задачи